In [13]:
import zipfile
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [2]:
zip_path = r"C:\Users\minha\Downloads\archive (2).zip"

images = []
labels = []

with zipfile.ZipFile(zip_path, 'r') as z:
    # Read CSV directly from zip
    with z.open("Train.csv") as f:  # <-- yahan Train.csv hona zaroori hai
        df = pd.read_csv(f)
    
    for i, row in df.iterrows():
        img_path = row['Path']  # Path column me image ka folder+name hota hai
        class_id = int(row['ClassId'])

        img_data = z.read(img_path)  # read image bytes
        img_array = np.frombuffer(img_data, np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (32, 32))
        images.append(img)
        labels.append(class_id)

In [3]:
images = np.array(images) / 255.0
labels = to_categorical(np.array(labels), 43)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [5]:
print("Train shape:", x_train.shape, y_train.shape)
print("Test shape:", x_test.shape, y_test.shape)

Train shape: (31367, 32, 32, 3) (31367, 43)
Test shape: (7842, 32, 32, 3) (7842, 43)


In [6]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

datagen.fit(x_train)


In [9]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(43, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


C:\Users\minha\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 30, 30, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 4, 4, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 43)                  │          11,051 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 235,627 (920.42 KB)

 Trainable params: 235,627 (920.42 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(datagen.flow(x_train, y_train, batch_size=64), 
                    epochs=15, 
                    validation_data=(x_test, y_test))


C:\Users\minha\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
491/491 ━━━━━━━━━━━━━━━━━━━━ 122s 231ms/step - accuracy: 0.3134 - loss: 2.3971 - val_accuracy: 0.5865 - val_loss: 1.2539
Epoch 2/15
491/491 ━━━━━━━━━━━━━━━━━━━━ 114s 233ms/step - accuracy: 0.6586 - loss: 1.0535 - val_accuracy: 0.8465 - val_loss: 0.4539
Epoch 3/15
491/491 ━━━━━━━━━━━━━━━━━━━━ 110s 224ms/step - accuracy: 0.8065 - loss: 0.5942 - val_accuracy: 0.9361 - val_loss: 0.2367
Epoch 4/15
491/491 ━━━━━━━━━━━━━━━━━━━━ 118s 241ms/step - accuracy: 0.8755 - loss: 0.3941 - val_accuracy: 0.9660 - val_loss: 0.1375
Epoch 5/15
491/491 ━━━━━━━━━━━━━━━━━━━━ 119s 243ms/step - accuracy: 0.9123 - loss: 0.2825 - val_accuracy: 0.9674 - val_loss: 0.1292
Epoch 6/15
491/491 ━━━━━━━━━━━━━━━━━━━━ 153s 265ms/step - accuracy: 0.9319 - loss: 0.2195 - val_accuracy: 0.9791 - val_loss: 0.0749
Epoch 7/15
491/491 ━━━━━━━━━━━━━━━━━━━━ 124s 251ms/step - accuracy: 0.9425 - loss: 0.1855 - val_accuracy: 0.9842 - val_loss: 0.0604
Epoch 8/15
491/491 ━━━━━━━━━━━━━━━━━━━━ 145s 295ms/step - accuracy: 0.9515 -

In [15]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Accuracy:", test_acc)

246/246 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.9949 - loss: 0.0177
Test Accuracy: 0.9948992729187012


In [16]:
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

246/246 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step


In [17]:
cm = confusion_matrix(y_true, y_pred_classes)

In [ ]:


plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=False, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
import zipfile

zip_path = r"C:\Users\minha\Downloads\archive (2).zip"

with zipfile.ZipFile(zip_path, 'r') as z:
    files = z.namelist()
    for f in files[:50]:  # pehle 50 file paths print karna
        print(f)
